In [198]:
with open("./data/Day 23/example.txt") as file:
    example = file.read()

In [199]:
with open('./data/Day 23/input.txt') as file:
    data = file.read()

In [200]:

def parse_input(inputstr):
    grid = []
    for line in inputstr.splitlines():
        grid.append(list(line))
    return grid

In [201]:
import networkx as nx
def build_graph(grid):
    G = nx.DiGraph()
    for y, row in enumerate(grid):
        for x, char in enumerate(row):
            if y == 0 and char != "#":
                start = (x, y)
            if y == len(grid) - 1 and char != "#":
                end = (x, y)
            if char == '#':
                continue
            G.add_node((x, y))
            for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)):
                if y + dy < 0 or y + dy >= len(grid):
                    continue
                if x + dx < 0 or x + dx >= len(row):
                    continue
                if grid[y + dy][x + dx] == '#':
                    continue
                if grid[y + dy][x + dx] == '^':
                    if dy == -1:
                        G.add_edge((x, y), (x + dx, y + dy))
                        continue
                    else:
                        continue
                if grid[y + dy][x + dx] == 'v':
                    if dy == 1:
                        G.add_edge((x, y), (x + dx, y + dy))
                        continue
                    else:
                        continue
                if grid[y + dy][x + dx] == '<':
                    if dx == -1:
                        G.add_edge((x, y), (x + dx, y + dy))
                        continue
                    else:
                        continue
                if grid[y + dy][x + dx] == '>':
                    if dx == 1:
                        G.add_edge((x, y), (x + dx, y + dy))
                        continue
                    else:
                        continue
                G.add_edge((x, y), (x + dx, y + dy))
    return G, start, end

# Part 1

In [202]:
G_example, start, end = build_graph(parse_input(example))
[len(path) - 1 for path in nx.all_simple_paths(G_example, start, end)]

[90, 74, 82, 86, 94, 82]

In [203]:
G, start, end = build_graph(parse_input(data))
max([len(path) - 1 for path in nx.all_simple_paths(G, start, end)])

2362

# Part 2

In [214]:
import networkx as nx


def build_graph_part2(grid):
    G = nx.Graph()
    for y, row in enumerate(grid):
        for x, char in enumerate(row):
            if y == 0 and char != "#":
                start = (x, y)
            if y == len(grid) - 1 and char != "#":
                end = (x, y)
            if char == "#":
                continue
            G.add_node((x, y))
            for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)):
                if y + dy < 0 or y + dy >= len(grid):
                    continue
                if x + dx < 0 or x + dx >= len(row):
                    continue
                if grid[y + dy][x + dx] == "#":
                    continue
                if G.has_edge((x, y), (x + dx, y + dy)) or G.has_edge(
                    (x + dx, y + dy), (x, y)
                ):
                    continue
                G.add_edge((x, y), (x + dx, y + dy), weight=1)
    return G, start, end

In [215]:
G_example, start, end = build_graph_part2(parse_input(example))
[len(path) - 1 for path in nx.all_simple_paths(G_example, start, end)]

[154, 118, 90, 118, 74, 82, 126, 86, 94, 150, 110, 82]

In [220]:
import networkx as nx


def simplify_graph(G, start, end):
    """Simplify graph by combining nodes with just a simple path between them to a single node"""
    simplified_graph = G.copy()
    for node in G.nodes:
        if start == node:
            continue
        if end == node:
            continue
        if node in simplified_graph:
            neighbors = list(simplified_graph.neighbors(node))

            if len(neighbors) == 1 and node != start and node != end:
                simplified_graph.remove_node(node)
            if len(neighbors) == 0:
                simplified_graph.remove_node(node)
                
            if len(neighbors) == 2:
                left_neighbor, right_neighbor = neighbors
                curr_weight = (
                    simplified_graph.edges[left_neighbor, node].get("weight")
                    + simplified_graph.edges[node, right_neighbor].get("weight")
                )
                simplified_graph.add_edge(left_neighbor, right_neighbor, weight=curr_weight)
                simplified_graph.remove_node(node)

    return simplified_graph

In [224]:
G, start, end = build_graph_part2(parse_input(data))
G_new = simplify_graph(G, start, end)

In [221]:
G_example, ex_start, ex_end = build_graph_part2(parse_input(example))
simplified_example_graph = simplify_graph(G_example, ex_start, ex_end)

In [225]:
ex_answer = [
    nx.path_weight(simplified_example_graph, path, weight="weight")
    for path in nx.all_simple_paths(simplified_example_graph, ex_start, ex_end)
]
max(ex_answer)

154

In [226]:
answer = ([
    nx.path_weight(G_new, path, weight="weight")
    for path in nx.all_simple_paths(G_new, start, end)
])
max(answer)

6538